In [30]:
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields.utils import scale
from numpy import pi, sqrt
import matplotlib.pyplot as plt
import numpy as np

# Coherent State

In [36]:
def abssqr(z):
    r"""
    Given :math:`z` returns :math:`|z|^2`.
    """
    return z.real**2 + z.imag**2

def entry(n, alpha):
        """coherent summation term"""
        return alpha**n / sqrt(fac(n))

In [69]:
cutoff_dim = 5
trunc = cutoff_dim

x_ini = 1
p_ini = 0.5
alpha = x_ini + p_ini * 1j
C = Coherent(a = alpha) 
z = C.p[0] * exp(1j * C.p[1]) # p[0]:alpha, p[1]:phase rotation

#print(C.p[0].x) # p[0] = Parameter(alpha) 
#print(C.p[1].x) # p[1] = Parameter(p=0)

print('alpha =',z.x)
fock_coherent =exp(-abssqr(z.x) / 2) * np.array([entry(n, z.x) for n in range(trunc)])
N = [print('basis:', i, fock_coherent[i]) for i in range(len(fock_coherent))]

alpha = (1+0.5j)
basis: 0 (0.5352614285189903+0j)
basis: 1 (0.5352614285189903+0.26763071425949514j)
basis: 2 (0.28386523936003233+0.3784869858133765j)
basis: 3 (0.054629890786030755+0.30046439932316915j)
basis: 4 (-0.04780115443777691+0.16388967235809226j)


# Squeezing Gate

squeezed_parity  
$$
        \sigma_{N,k} = \begin{cases}
            (N-k)/2, & \text{mod}(N-k,2) \neq 0\\
            0, &\text{otherwise}
        \end{cases}
$$

$$
            prefac_{N,n,k} = \frac{\sigma_{N,k}\sqrt{n!N!}}
            {k!\left(\frac{n-k}{2}\right)!\left(\frac{N-k}{2}\right)!}
$$

In [70]:
from scipy.special import gammaln as lg

def squeeze_parity(D):
    r"""Creates the parity prefactor needed for squeezing in the Fock basis.

    .. math::
        \text{\sigma}_{N,k} = \begin{cases}
            (N-k)/2, & \text{mod}(N-k,2) \neq 0\\
            0, &\text{otherwise}
        \end{cases}

    Args:
        D (numpy.array): generate the prefactors for a Fock truncation of :math:`D`.
    """
    k = np.int(np.ceil(D/4) * 4)
    v = np.full(k, 1)
    v[1::2] = 0
    v[2::4] = -1
    v = np.vstack([np.roll(v, i) for i in range(k)])
    return v[:D, :D]

def generate_squeeze_factors(D):
    r"""Generate squeezing factors in the Fock basis.

    This function generates the squeezing prefactors,

        .. math::
            prefac_{N,n,k} = \frac{\sigma_{N,k}\sqrt{n!N!}}
            {k!\left(\frac{n-k}{2}\right)!\left(\frac{N-k}{2}\right)!}

    where :math:`\sigma_{N,k}` is the parity, given by :func:`~.squeeze_parity`.

    Args:
        D (int): generate prefactors for :math:`D` dimensions.
    """
    dim_array = np.arange(D)
    N = dim_array.reshape((-1, 1, 1))
    n = dim_array.reshape((1, -1, 1))
    k = dim_array.reshape((1, 1, -1))

    # we only perform the sum when n+N is divisible by 2
    # in which case we sum 0 <= k <= min(N,n)
    mask = np.logical_and((n+N)%2 == 0, k <= np.minimum(N, n))

    # need to use np.power to avoid taking the root of a negative
    # in the numerator (these are discarded by the mask anyway)
    signs = squeeze_parity(D).reshape([D, 1, D])
    logfac = np.where(mask, 0.5*(lg(n+1) + lg(N+1)) - lg(k+1) - lg((n-k)/2+1) - lg((N-k)/2+1), 0)

    if D <= 600:
        prefactor = np.exp(logfac, dtype=np.float64)*signs*mask
    else:
        prefactor = np.exp(logfac, dtype=np.float128)*signs*mask

    return prefactor

In [78]:
r = -2
theta = 0
S = Sgate(r)
z = S.p[0] * exp(1j * S.p[1])
print(z.x)

dim_array = np.arange(trunc)
N = dim_array.reshape((-1, 1, 1))
n = dim_array.reshape((1, -1, 1))
k = dim_array.reshape((1, 1, -1))
print(N.shape)
print(n.shape)
print(k.shape)
mask = np.logical_and((n+N)%2 == 0, k <= np.minimum(N, n))
print(mask.shape)

(-2+0j)
(5, 1, 1)
(1, 5, 1)
(1, 1, 5)
(5, 5, 5)


In [96]:
# perform the summation over k
scale = mask * np.power(np.sinh(r)/2, mask*(N+n-2*k)/2) / (np.cosh(r)**((N+n+1)/2))
ph = exp(1j*theta*(N-n)/2) # rotate
prefac = generate_squeeze_factors(trunc)
ret = np.sum(scale*ph*prefac, axis=-1)
ret

array([[ 0.51556011+0.j,  0.        +0.j, -0.35144209+0.j,
         0.        +0.j,  0.29340929+0.j],
       [ 0.        +0.j,  0.13703703+0.j,  0.        +0.j,
        -0.16179795+0.j,  0.        +0.j],
       [ 0.35144209+0.j,  0.        +0.j, -0.20314294+0.j,
         0.        +0.j,  0.13918841+0.j],
       [ 0.        +0.j,  0.16179795+0.j,  0.        +0.j,
        -0.18135109+0.j,  0.        +0.j],
       [ 0.29340929+0.j,  0.        +0.j, -0.13918841+0.j,
         0.        +0.j,  0.06800103+0.j]])

# BS Gate

$$
B(\theta,\phi) = \exp\left(\theta (e^{i \phi} a^\dagger b -e^{-i \phi}a b^\dagger) \right)
$$

In [114]:
from cmath import phase

BS = BSgate(pi/4, 0)
t = cos(BS.p[0].x)
r = sin(BS.p[0].x) * exp(1j * BS.p[1].x)
print('Transmit:', t, '| Reflect:' ,r)

Transmit: 0.7071067811865476 | Reflect: (0.7071067811865475+0j)


$$
prefac_{N,n,M,m,k} = (-1)^{N-k}\sqrt{\binom{n}{k}\binom{m}{N-k}\binom{N}{k}\binom{M}{n-k}}
$$

In [125]:
import itertools
from scipy.special import binom

def generate_bs_factors(D):
    r"""Generate beamsplitter factors in the Fock basis.

    This function generates the beamsplitter prefactors,

        .. math::
            prefac_{N,n,M,m,k} = (-1)^{N-k}\sqrt{\binom{n,k}\binom{m,N-k}\binom{N,k}\binom{M,n-k}}

    for a specific cutoff dimension :math:`D`.

    Note that the last dimension will only contain non-zero values
    for indices ``0`` to ``n``.

    Args:
        D (int): generate prefactors for :math:`D` dimensions.
    """
    prefac = np.zeros([D]*5)

    for (N, M, n) in itertools.product(*([range(D)]*3)):
        m = N+M-n
        k = np.arange(n+1)
        if 0 <= m < D:
            # pylint: disable=bad-whitespace
            prefac[N,n,M,m,:n+1] = (-1.0)**(N-k) \
                * np.sqrt(binom(n, k)*binom(m, N-k)*binom(N, k)*binom(M, n-k))

    return prefac

In [128]:
dim_array = np.arange(trunc)
N = dim_array.reshape((-1, 1, 1, 1, 1))
n = dim_array.reshape((1, -1, 1, 1, 1))
M = dim_array.reshape((1, 1, -1, 1, 1))
k = dim_array.reshape((1, 1, 1, 1, -1))

tpwr = M-n+2*k
rpwr = n+N-2*k
phi = phase(r)

T = np.power(t, tpwr) if t != 0 else np.where(tpwr != 0, 0, 1)
R = np.power(r, rpwr) if r != 0 else np.where(rpwr != 0, 0, 1)
prefac = generate_bs_factors(trunc)

BS = np.sum(exp(-1j*(pi+phi)*(n-N)) * T * R * prefac[:trunc,:trunc,:trunc,:trunc,:trunc], axis=-1)
BS = BS.swapaxes(0, 1).swapaxes(2, 3)

BS.shape

(5, 5, 5, 5)

# Measure X

In [ ]:
MX = MeasureX() # equal to MeasureHomodyne(0)
